# import libraries:

In [1]:
import pandas as pd
import numpy as np
import random

# get inputs:

In [2]:
min_v = float(input("Enter the minimum value (min_v) you want to achieve: "))
max_w = float(input("Enter the maximum weight (max_w) allowed: "))

min_n = int(input("Enter the minimum number of snack types (min_n): "))
max_n = int(input("Enter the maximum number of snack types (max_n): "))


Enter the minimum value (min_v) you want to achieve:  22
Enter the maximum weight (max_w) allowed:  11
Enter the minimum number of snack types (min_n):  1
Enter the maximum number of snack types (max_n):  3


In [3]:
population_size = int(input("Enter population size: "))

Enter population size:  150


In [4]:
max_run = int(input())

 100


# read CSV file and save to Data Frame:

In [5]:

file_path = 'snacks.csv'
df = pd.read_csv(file_path)


In [6]:
print(df)

            Snack  Available Weight  Value
0          MazMaz                10     10
1   Doogh-e-Abali                15     10
2            Nani                 5      5
3            Jooj                 7     15
4         Hot-Dog                20     15
5           Chips                 8      6
6        Nooshaba                12      8
7        Shokolat                 6      7
8       Chocoroll                 9     12
9         Cookies                11     11
10        Abnabat                 4      4
11   Adams-Khersi                14      9
12        Popcorn                16     13
13         Pastil                 3      7
14       Tordilla                10      9
15       Masghati                 5      6
16        Ghottab                 7     10
17   Saghe-Talaei                 9     11
18    Choob-Shoor                13     12


# Part One: Defining Basic Concepts

## Gene:

In [7]:
#Gene:
class Gene:
    def __init__(self, name, max_weight, value_per_weight):
        self.name = name
        self.max_weight = max_weight
        self.weight = random.uniform(0, self.max_weight)
        self.value_per_weight = value_per_weight
        
    def __repr__(self):
        return f"Gene(name={self.name}, weight={self.weight}, max_weight={self.max_weight}, value_per_weight={self.value_per_weight})\n"

    def __str__(self):
        return f"{self.name}: Weight={self.weight}, Value per Weight={self.value_per_weight}"


In [8]:
class Chromosome:
    def __init__(self, geness):
        self.genes_temp = geness
        self.genes = []
        for g in self.genes_temp:
            self.genes.append(Gene(g.name, g.max_weight, g.value_per_weight))
        for gene in self.genes:
            gene.weight = random.uniform(0, gene.max_weight)
        self.genes.sort(key=lambda x: x.name)
        self.total_weight = sum(gene.weight for gene in self.genes)
        self.total_value = sum(gene.weight * gene.value_per_weight for gene in self.genes)
        self.variety_of_snacks = len(self.genes)
        self.fitness = 0 
         
    
    def __repr__(self):
        return f"\nChromosome(genes=\n{self.genes}, fitness={self.fitness}) \nTotal Weight: {self.total_weight}\nTotal Value: { self.total_value}\nRange: {self.variety_of_snacks}\n"
        
    def __str__(self):
        genes_str = '\n'.join(str(gene) for gene in self.genes)
        return f"\nChromosome Details:\nGenes:\n{genes_str}\nFitness: {self.fitness} \nTotal Weight: {self.total_weight}\nTotal Value: { self.total_value}\nRange: {self.variety_of_snacks}\n"
        



## Genes pool:

In [9]:

genes_pool = df.copy()
genes_pool.rename(columns={'Snack': 'Name'}, inplace=True)
genes_pool.rename(columns={'Available Weight': 'Maximum Weight'}, inplace=True)
genes_pool['Value per Weight'] = genes_pool['Value'] / genes_pool['Maximum Weight']
genes_pool = genes_pool[['Name', 'Maximum Weight', 'Value per Weight']]

gene_objects = [Gene(row['Name'], row['Maximum Weight'], row['Value per Weight']) for index, row in genes_pool.iterrows()]


In [10]:
print(genes_pool)

             Name  Maximum Weight  Value per Weight
0          MazMaz              10          1.000000
1   Doogh-e-Abali              15          0.666667
2            Nani               5          1.000000
3            Jooj               7          2.142857
4         Hot-Dog              20          0.750000
5           Chips               8          0.750000
6        Nooshaba              12          0.666667
7        Shokolat               6          1.166667
8       Chocoroll               9          1.333333
9         Cookies              11          1.000000
10        Abnabat               4          1.000000
11   Adams-Khersi              14          0.642857
12        Popcorn              16          0.812500
13         Pastil               3          2.333333
14       Tordilla              10          0.900000
15       Masghati               5          1.200000
16        Ghottab               7          1.428571
17   Saghe-Talaei               9          1.222222
18    Choob-

In [11]:
print(gene_objects)

[Gene(name=MazMaz, weight=9.742901788652798, max_weight=10, value_per_weight=1.0)
, Gene(name=Doogh-e-Abali, weight=0.4706584610703801, max_weight=15, value_per_weight=0.6666666666666666)
, Gene(name=Nani, weight=3.9076236184654607, max_weight=5, value_per_weight=1.0)
, Gene(name=Jooj, weight=0.863928049005298, max_weight=7, value_per_weight=2.142857142857143)
, Gene(name=Hot-Dog, weight=17.16478003948783, max_weight=20, value_per_weight=0.75)
, Gene(name=Chips, weight=5.198964707681567, max_weight=8, value_per_weight=0.75)
, Gene(name=Nooshaba, weight=2.5026383940088457, max_weight=12, value_per_weight=0.6666666666666666)
, Gene(name=Shokolat, weight=2.980997985585362, max_weight=6, value_per_weight=1.1666666666666667)
, Gene(name=Chocoroll, weight=5.394801718711647, max_weight=9, value_per_weight=1.3333333333333333)
, Gene(name=Cookies, weight=10.651429680719918, max_weight=11, value_per_weight=1.0)
, Gene(name=Abnabat, weight=1.4343516785082344, max_weight=4, value_per_weight=1.0)
,

# Part Two: Primary population production

In [12]:
initial_population = []
for _ in range(population_size):
    x = random.randint(min_n, max_n)
    genes_temp = random.sample(gene_objects, x)
    c = Chromosome(genes_temp)
    initial_population.append(c)
    c = None


In [13]:
initial_population

[
 Chromosome(genes=
 [Gene(name=Hot-Dog, weight=9.618904185065173, max_weight=20, value_per_weight=0.75)
 ], fitness=0) 
 Total Weight: 9.618904185065173
 Total Value: 7.2141781387988795
 Range: 1,
 
 Chromosome(genes=
 [Gene(name=Saghe-Talaei, weight=5.116389530020273, max_weight=9, value_per_weight=1.2222222222222223)
 ], fitness=0) 
 Total Weight: 5.116389530020273
 Total Value: 6.253364981135889
 Range: 1,
 
 Chromosome(genes=
 [Gene(name=Adams-Khersi, weight=5.21525493756406, max_weight=14, value_per_weight=0.6428571428571429)
 , Gene(name=Doogh-e-Abali, weight=3.154908782456232, max_weight=15, value_per_weight=0.6666666666666666)
 , Gene(name=MazMaz, weight=6.401840695046756, max_weight=10, value_per_weight=1.0)
 ], fitness=0) 
 Total Weight: 14.772004415067048
 Total Value: 11.857777105118283
 Range: 3,
 
 Chromosome(genes=
 [Gene(name=Abnabat, weight=3.561881137058269, max_weight=4, value_per_weight=1.0)
 , Gene(name=Chocoroll, weight=5.175178717372866, max_weight=9, value_per

# Part three: Implementation and specification of compatibility criterion function

## fitness

In [14]:
def calculate_fitness(chromosome):
    fitness = chromosome.total_value
    penalty_weight =max_w - chromosome.total_weight
    penalty_value = chromosome.total_value - min_v
    #penalty_varity = min(chromosome.variety_of_snacks - max_n , min_n - chromosome.variety_of_snacks)
    penalties = 0
    if penalty_weight < 0:
        penalties = penalty_weight 
    if penalty_value < 0:
        penalties = penalties + penalty_value 
    if penalties < 0 :
        fitness = penalties
    return fitness
    


In [15]:
#Update Chromosomes Fitness'
def update_fitness(pop):
    for chromosome in pop:
        chromosome.fitness = calculate_fitness(chromosome)
    return pop
    

In [16]:
initial_population = update_fitness(initial_population)

## find winner

In [17]:
def find_winner(population):
    max_fitness = float('-inf')
    winner = population[0];
    for chromosome in population:
        if (chromosome.fitness > max_fitness):
            max_fitness = chromosome.fitness
            winner = chromosome
    return winner

In [18]:
def check_for_answer(population):
    winner = find_winner(population)
    if winner.fitness >= min_v:
        return winner
    return None

# Part four: Generating a new generation

## Probability:

In [19]:
prop_c = 0.3
def decide_with_probability(p, thing1, thing2):
    if random.random() < p:
        return thing1
    else:
        return thing2

## Crossover:

In [20]:
def crossover(prob , parent1, parent2, min_n, max_n):
    x = min(len(parent1.genes), len(parent2.genes)) - 1
    if x <= 1:
        return [parent1, parent2]
    cross_point =  random.randint(1, x)
    offspring_genes = parent1.genes + parent2.genes

    counter = 0
    def remove_dupplicate(objects1, objects2, counter):
        def has_duplicate(objects):
            seen_names = set()
            for obj in objects:
                obj_name = obj.name
                if obj_name in seen_names:
                    return obj
                seen_names.add(obj_name)
            return None

        while True and counter < 10000:
            counter = counter + 1
            dup1 = has_duplicate(objects1.genes)
            dup2 = has_duplicate(objects2.genes)
            if dup1 != None:
                if dup2 != None:
                    index1 = objects1.genes.index(dup1)
                    index2 = objects2.genes.index(dup2)
                    objects1.genes.pop(index1)
                    objects2.genes.pop(index2)
                    objects1.genes.append(dup2)
                    objects2.genes.append(dup1)
                else :
                    index1 = objects1.genes.index(dup1)
                    objects1.genes.pop(index1)
                    new = objects2.genes.pop()
                    objects1.genes.append(new)
                    objects2.genes.append(dup1)
                    
            elif dup2 != None:
                    index2 = objects2.genes.index(dup2)
                    objects2.genes.pop(index2)
                    new = objects1.genes.pop()
                    objects2.genes.append(new)
                    objects1.genes.append(dup2)
            else:
                    break;
        return [objects1, objects2], counter 
                
    
                
    
    #offspring1_genes = random.sample(offspring_genes,  random.randint(min_n, max_n))
    #offspring2_genes = random.sample(offspring_genes,  random.randint(min_n, max_n))
    offspring1_genes = parent1.genes[:cross_point] + parent2.genes[cross_point:]
    offspring2_genes = parent2.genes[:cross_point] + parent1.genes[cross_point:]
    
    child1 = Chromosome(offspring1_genes);
    child2 = Chromosome(offspring2_genes);

    [child1, child2] , counter = remove_dupplicate(child1, child2, counter)
    if counter == 10000: 
        return  [parent1, parent2]
    def select_new_generation(prob, child1, child2, parent1, parent2):
        fitness_child1 = calculate_fitness(child1)
        fitness_child2 = calculate_fitness(child2)
        sum_fitness_children = fitness_child1 + fitness_child2
        sum_fitness_parents = parent1.fitness + parent2.fitness

        #if sum_fitness_parents > sum_fitness_children:
        #    return decide_with_probability(prob, [child1, child2], [parent1, parent2])
        return decide_with_probability(prob,  [parent1, parent2],  [child1, child2])

    return select_new_generation(prob, child1, child2, parent1, parent2)
    

def generate_new_population_crossover(prob, population):
    np.random.shuffle(population)
    new_population = []
    for i  in range(0, round((population_size)/2)):
        parent1 = population[i]
        parent2 = population[-i]
        new_generation = crossover(prob , parent1, parent2, min_n, max_n)
        new_population.append(new_generation[0])
        new_population.append(new_generation[1])
    return new_population




## Mutation:

In [21]:

def mutation(prob_m, population):
    np.random.shuffle(population)
    new_population = []
    for i  in range(0, population_size):
        parent = population[i]
        genes = []
        for g in parent.genes:
            genes.append(g)

        genes.sort(key=lambda x: x.value_per_weight)

        pre_gene = genes.pop(0)

        rand_gene = gene_objects
        rand_gene.sort(key=lambda x: x.value_per_weight, reverse=True)

        new_gene = pre_gene
        for g in rand_gene:
            if pre_gene.name == g.name:
                break
            elif any(x.name == g.name for x in parent.genes) == False:
                continue 
            else:
                new_gene = g
                break

        new_gene.weight = pre_gene.weight
        if any(x.name == new_gene.name for x in genes) == True:
            genes.append(new_gene)
        if len(genes) == 0:
            return population
        child = Chromosome(genes)
       # print (f"child : {child}")
        genes.clear()
        
        new_generation = decide_with_probability(prob_m, parent, child)
        new_population.append(new_generation)
    return new_population

## Algorithm:

In [22]:
population = initial_population

winner = check_for_answer(initial_population)
counter = 0
cur_fit = sum(c.fitness for c in population)
pre_fit = sum(c.fitness for c in population)
while check_for_answer(population) == None and counter < 1000:
    counter = counter + 1
    prob_m = min (1 / (counter + 1), 2 /max_run)
    if counter > 2: 
        if (cur_fit >= pre_fit) :
            prob_m = 0

    pre_fit = cur_fit
    new_population = population
    new_population = generate_new_population_crossover(prop_c, population)
    new_population = mutation(prob_m, new_population)
    
    population = update_fitness(new_population)
    cur_fit = sum(c.fitness for c in population)
    winner = check_for_answer(population)
    print(counter)
    print(cur_fit, pre_fit)
    print (max(c.fitness for c in population))
    #print(population)
    print("______________________________________________________")
    
    

1
-2236.853974433552 -2229.6821730783163
-3.491824996197739
______________________________________________________
2
-2228.206445464369 -2236.853974433552
-3.491824996197739
______________________________________________________
3
-2236.957440391603 -2228.206445464369
-3.491824996197739
______________________________________________________
4
-2228.4527461661837 -2236.957440391603
-3.491824996197739
______________________________________________________
5
-2215.336954749558 -2228.4527461661837
-3.491824996197739
______________________________________________________
6
-2206.957858435458 -2215.336954749558
-3.491824996197739
______________________________________________________
7
-2207.1827406379994 -2206.957858435458
-3.491824996197739
______________________________________________________
8
-2216.756744085106 -2207.1827406379994
-2.9044838382257403
______________________________________________________
9
-2219.7076355710205 -2216.756744085106
-2.6399298118221015
_____________________

In [23]:
population


[
 Chromosome(genes=
 [Gene(name=Chips, weight=0.5894791927109422, max_weight=8, value_per_weight=0.75)
 , Gene(name=Jooj, weight=2.345649130050902, max_weight=7, value_per_weight=2.142857142857143)
 , Gene(name=Tordilla, weight=2.4464139540524146, max_weight=10, value_per_weight=0.9)
 ], fitness=-14.329727053853404) 
 Total Weight: 5.381542276814258
 Total Value: 7.670272946146597
 Range: 3,
 
 Chromosome(genes=
 [Gene(name=Adams-Khersi, weight=0.6702397354507954, max_weight=14, value_per_weight=0.6428571428571429)
 , Gene(name=Shokolat, weight=5.799357382760811, max_weight=6, value_per_weight=1.1666666666666667)
 ], fitness=-14.803214652084495) 
 Total Weight: 6.469597118211606
 Total Value: 7.1967853479155055
 Range: 2,
 
 Chromosome(genes=
 [Gene(name=Jooj, weight=0.033035997408193074, max_weight=7, value_per_weight=2.142857142857143)
 ], fitness=-21.929208576982443) 
 Total Weight: 0.033035997408193074
 Total Value: 0.07079142301755659
 Range: 1,
 
 Chromosome(genes=
 [Gene(name=C

In [24]:
def print_winner(winner):
    for Gene in winner.genes:
        print(f"{Gene.name}: {Gene.weight}")
    print(f"Total Weight: {winner.total_weight}")
    print(f"Total Value: {winner.total_value}")

In [25]:
print_winner(winner)

AttributeError: 'NoneType' object has no attribute 'genes'